In [134]:
#!pip install pdfplumber
import os
import pdfplumber
import re
from re import search
from sklearn.cluster import KMeans
import sklearn.metrics
import pandas as pd
import csv

In [138]:
#path = r'H:\\Dropbox\\Personale\\Ricerca\\XAI\\00. A.Gatti\\Quantitative AI\\GreenAI\\Articolo\\Bibliografia\TUTTI'
path = r'c:\temp'

In [139]:
articles = []
titles=[]
print(path)
for filename in os.listdir(path):
    if filename.endswith('.pdf'):
        titles.append(filename)
        fullpath = os.path.join(path, filename)
        print(fullpath)
        all_text = ""
        with pdfplumber.open(fullpath) as pdf:
            for page in pdf.pages:
                text = page.extract_text().lower()
                #print(text)
                all_text += '\n' + text
        articles.append(all_text)

c:\temp
c:\temp\07.Sustainability challenges of Artificial Intelligence and Policy Implications.pdf
c:\temp\08.SustainableAIEnvironmentalImplicationsChallengesandOpportunities.pdf
c:\temp\09. GreenAlgorithmsQuantifyingthecarbonfootprintofcomputation.pdf
c:\temp\A Systematic Review of Green AI.pdf
c:\temp\Diversification in the age of the 4th industrial revolution_ The role of artificial intelligence_ green bonds and cryptocurrencies.pdf
c:\temp\Emerging technologies based on artificial intelligence to assess the quality and consumer preference of beverages.pdf
c:\temp\Towards green automated machine learning_ Status quo and future directions.pdf
c:\temp\__a smart framework for supplying the biogas energy in green buildings using an integration of response surface methodology_ artificial intelligence and petri net__.pdf


In [140]:
corpus = pd.DataFrame(list(zip(titles,articles)),columns=['Article','Text'])
corpus.to_csv("corpus.csv")

In [ ]:
# Aggiungere il titolo al testo

In [141]:
corpus

,Article,Text
0,07.Sustainability challenges of Artificial Int...,\nsustainability of digitalisation\ndiscussing...
1,08.SustainableAIEnvironmentalImplicationsChall...,\n1\nsustainable ai: environmental implication...
2,09. GreenAlgorithmsQuantifyingthecarbonfootpri...,\ngreen algorithms: quantifying the carbon\nfo...
3,A Systematic Review of Green AI.pdf,\na systematic review of green ai\nrobertoverd...
4,Diversification in the age of the 4th industri...,"\ncitation:\nhuynh, tld and hille, e and nasir..."
5,Emerging technologies based on artificial inte...,\nbeverages\nreview\nemerging technologies bas...
6,Towards green automated machine learning_ Stat...,\njournalofartificialintelligenceresearch77(20...
7,__a smart framework for supplying the biogas e...,\n1 constructing a novel smart framework for s...


In [78]:
#regex = re.compile('[^a-zA-Z]')
regex = re.compile('a-zA-Z0-9_]')

for index in range(len(articles)):
    if search('abstract', articles[index]):
        articles[index] = articles[index].split('abstract')[1]

    #First parameter is the replacement, second parameter is your input string
    articles[index] = regex.sub('', articles[index])

In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.8,         # drop words that occur in more than X percent of documents
                             #min_df=8,      # only use words that appear at least X times
                             use_idf=True,   # Use idf
                             norm=u'l2',     # Normalization
                             smooth_idf=True # Prevents divide-by-zero errors
                       )

X = vectorizer.fit_transform(articles)

In [129]:
K = range(2,20)
max = -1
clusters = 2
for k in K:
    kmeanModel = KMeans(n_clusters=k, init='k-means++', 
            max_iter=1000, n_init=1, verbose=0, random_state=3425)
    kmeanModel.fit(X)
    score = round(sklearn.metrics.silhouette_score(X, kmeanModel.labels_, metric='euclidean', random_state=42),5)
    print(k, 'Silhouette Score:', score)
    if max < score:
        max = score
        clusters = k

print("Clusters:", clusters)

2 Silhouette Score: 0.0192
3 Silhouette Score: 0.02285
4 Silhouette Score: 0.02412
5 Silhouette Score: 0.02656
6 Silhouette Score: 0.02778
7 Silhouette Score: 0.0261
8 Silhouette Score: 0.02013
9 Silhouette Score: 0.01874
10 Silhouette Score: 0.01606
11 Silhouette Score: 0.01238
12 Silhouette Score: 0.01312
13 Silhouette Score: 0.01309
14 Silhouette Score: 0.01288
15 Silhouette Score: 0.01357
16 Silhouette Score: 0.00911
17 Silhouette Score: 0.01289
18 Silhouette Score: 0.01323
19 Silhouette Score: 0.01366
Clusters: 6


In [130]:
true_k = clusters
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=1000, n_init=10)
model.fit(X)
labels=model.labels_
labels

array([3, 4, 2, 4, 5, 1, 4, 5, 1, 4, 4, 5, 1, 5, 4, 4, 1, 0, 4, 4, 4, 1,
       0, 4, 4, 3, 4, 4, 3, 1, 4, 4, 4, 0, 1, 1, 1, 1, 1, 4, 1, 4, 4, 4,
       4, 1, 1, 4, 4, 5, 4, 4, 4, 4, 4, 3, 4, 1, 0, 1, 1, 5, 4, 1, 0, 0,
       4, 5, 5, 1, 4, 1, 4, 0, 5, 1, 4, 4, 5, 2, 4, 0, 0, 1, 1, 2, 4, 4,
       5, 5, 1, 0, 4, 4, 4, 0, 4, 1, 1, 4, 1, 4, 0, 5, 0, 4, 4, 3, 5, 3,
       0, 4, 4, 4, 3, 5, 1, 5])

In [131]:
df = pd.DataFrame(list(zip(titles,labels)),columns=['Article','Cluster'])
df_s= df.sort_values(by=['Cluster'])
df_s.to_csv("clusters.csv")